**RTSA 2023**

*Replace this line with your name.*

# Project C: Non-LTE Line Formation

#### Header and imports

In [4]:
# Modify this cell to customise your imports

### Exercise 1: The Feautrier method and the $\mathbf{\Lambda}$ operator [30 points]

The Feautrier method consists of solving the equation

\begin{equation}
\mathbf{T}P = S,
\end{equation}

where $\mathbf{T}$ is a tridiagonal matrix. The function `Tmatrix` computes $\mathbf{T}$ for a single frequency and direction on a 1D plane-parallel atmosphere, given an optical depth scale and $\mu$. Below we use the C/Python convention that 0 is the first index of an array, and $n-1$ the last point.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[6 points]* At the top of the atmosphere, we can assume $I^-=0$. Why? Using a Taylor expansion around the Feautrier $P_0$ and assuming $I^-=0$ at the top of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the top can be written as:

$$
\begin{aligned}
B_0 &= \frac{2}{\Delta\tau^2} + \frac{2}{\Delta\tau} + 1, \\ 
C_0 &= \frac{2}{\Delta\tau^2}, \\
\Delta\tau &=\tau_{1} - \tau_{0}.
\end{aligned}
$$ 

* *[7 points]* At the bottom of the atmosphere, we can assume $I^+=S$. Why? Using a Taylor expansion around the Feautrier $P_{n-1}$, and assuming $I^+=S$ at the bottom of the atmosphere, show that the boundary coefficients of $\mathbf{T}$ at the bottom can be written as:

$$
\begin{aligned}
A_{n-1} &= \frac{2}{\Delta\tau(\Delta\tau+2)},  \\ 
B_{n-1} &= \frac{2+2\Delta\tau +\Delta\tau^2}{\Delta\tau(\Delta\tau+2)},\\
\Delta\tau &=\tau_{n-1} - \tau_{n-2}.
\end{aligned}
$$

* *[8 points]* Using a Gaussian quadrature with 5 points, write a function called `solve_lambda_implicit()` to solve $J=\mathbf{\Lambda}S$ without explicitly building $\mathbf{\Lambda}$ as a matrix, taking as arguments $\tau$ and $S$. Do not use the `feautrier` function. Use it to solve the simple case where `tau = numpy.logspace(-4, 2, 50)` and $S=1$ at all depth points. Plot $J$ and $S$ vs $\log_{10}\tau$. Discuss the values of $J$ at $\tau\gg 1$ and $\tau\ll 1$. How do they compare with analytical solutions?

* *[9 points]* Now write a function called `solve_lambda_direct()` where you build the $\mathbf{\Lambda}$ matrix explicity and solve for $J$. Using `tau = numpy.logspace(-4, 2, 50)`, plot the $\mathbf{\Lambda}$ matrix and discuss its physical meaning. Explore the effect of using a different number of points in the Gaussian quadrature. What is a reasonable number of points?

</div>

In [4]:
# Your answers here

### Exercise 2: $\Lambda$ iteration for coherent scattering in a two-level atom [35 points]

The source function for a two-level atom with coherent scattering can be written as

\begin{equation}
S = (1-\varepsilon)\mathbf{\Lambda}[S] + \varepsilon B,
\end{equation}
and a direct solution can be written as
\begin{equation}
S = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda})^{-1}[\varepsilon B],
\end{equation}

which involves inverting matrices (impractical for most real-life applications). Other numerical methods to solve for $S$ are the classical $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
\end{equation}
Where we need a guess for $S^{(1)}$, often $S^{(1)}=B.$ An alternative that is much more efficient is the approximate (or accelerated) $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right].
\end{equation}

For the iterative schemes, we can measure the convergence speed by calculating the fractional difference between source functions of successive iterations, e.g.:
\begin{equation}
\delta = \left|\left|\frac{S_\nu^{(n+1)}-S_\nu^{(n)}}{S_\nu^{(n)}}\right| \right|.
\end{equation}

Throughout this exercise, use a Gaussian quadrature with 5 points as your angle quadrature.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[10 points]* On a similar vein to what you did in the previous exercise, write a function called `solve_cs_direct()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using a direct solution for the problem of coherent scattering in a two-level atom (using matrix inversions). Use it to solve the problem for $\varepsilon=10^{-3}$, `tau = numpy.logspace(-4, 4, 50)` and $B=1.5\tau$. Do a log-log plot for S, J, and B. In this case, why is $J>B$ at the surface?
    
* *[11 points]* Write a function `solve_cs_CLI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the classical $\Lambda$ iteration (CLI). For the simple case where $B=1$ everywhere, $\varepsilon=10^{-2}$, plot $S$, $J$, $B$, for CLI and the direct solution from the previous question. Does CLI work well for this case? How many iterations do you need to achieve $\delta < 10^{-3}$?
    
* *[14 points]* Write a function `solve_cs_ALI()` that takes as arguments $\tau$, $B$, and $\varepsilon$ and computes $S$ and $J$ using the accelerated $\Lambda$ iteration (ALI). For the $\mathbf{\Lambda}^*$ operator, use the OAB operator, which simply uses the diagonal of the full $\mathbf{\Lambda}$ operator. Usually, this $\mathbf{\Lambda}^*$ is computed without having to build the $\mathbf{\Lambda}$ explicitly, during the tridiagonal solution of $\mathbf{T}P = S$. Use the provided function `diag_inverse_tri()` to obtain $\mathbf{\Lambda}^*$ from the inverse of $\mathbf{T}$ matrices (as before, account for different angle quadratures), using `Tmatrix(tau, mu, format='banded')` to get $\mathbf{T}$ in the correct format. For the simple case where $\varepsilon=10^{-5}$, `tau = numpy.logspace(-4, 4, 50)`, `B = tau[::-1] * 1.5` , plot $S$ and $B$, for $S$ computed with the direct solution, CLI, and ALI, both with a maximum of 100 iterations. Discuss the differences. How many iterations of ALI do you need to be close to the direct solution? And how many iterations of CLI?
    
    
</div>

In [4]:
# Your answers here

### Exercise 3: Two-level non-LTE problem from a 1D atmosphere [30 points]

Now we will relax some of the previous approximations and work on a more realistic case: solving the non-LTE problem for a spectral line from a two-level atom in a 1D plane-parallel atmosphere. To avoid excessive coding, we will not solve the statistical equilibrium equations and will compute the line in a much more approximated fashion that you did in Project B. 

We will no longer assume coherent scattering, but instead complete redistribution, so that

$$
\begin{aligned}
S &= (1-\varepsilon)\overline{J} + \varepsilon B, \\
\overline{J} &= \overline{\mathbf{\Lambda}}\left[S\right]. \\
\end{aligned}
$$

You will solve the problem using ALI with a diagonal operator and a Feautrier solver. Since we have several frequencies, $\overline{\mathbf{\Lambda}}^*$ is now averaged over angle and frequency:

\begin{equation}
\overline{\mathbf{\Lambda}}^* = \frac{1}{2} \int_{-1}^{1}\int_0 ^ \infty \varphi(\nu-\nu_0) \mathbf{\Lambda}^*_{\mu\nu} \mathrm{d}\nu \mathrm{d}\mu,
\end{equation}

where $\varphi(\nu-\nu_0)$ is the line profile, and $\mathbf{\Lambda}^*_{\mu\nu}$ the diagonal operator for a given frequency and direction.

You will make use of the provided function `feautrier()`, which computes the Feautrie solution as you did in previous questions, but in a form that is numerically more stable. For a given $\tau$ and $S$, it returns the Feautrier $P_\mu\nu$ (if `doLstar=True`) and also $I^+$ and $I^-$ (if `doI=True`). As before, use a Gaussian quadrature with 5 points as your angle quadrature. Note that `feautrier()` works on a given ray and does not take $\mu$ as an argument. For inclined rays in the 1D plane-parallel case, you need to pass it $\tau/\mu$, where $\tau$ is your vertical optical depth scale.

We create $\varphi(\nu)$ from a Voigt profile. To facilitate the frequency integration of $\overline{\mathbf{\Lambda}}^*$, when we obtain the profile we also compute integration weights $w_i$ so that the frequency integration is a simple weighed sum:

$$
\int_0^\infty \varphi(\nu-\nu_0)\mathrm{d}\nu = \sum_i w_i \varphi_i = 1.
$$
Instead of writing a detailed function to compute the line extinction $\alpha_\nu^l$, we will just create $\alpha_\nu^l$ by scaling $\alpha^c$ by the line profile and an $\eta$ factor.

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* *[4 points]* Create a function called `profile()` that returns a Voigt profile $\varphi_\nu$ and integration weights $w$ as function of dimensionless frequency $u$. It should have as arguments `u_max` (the maximum range in dimensionless frequencies, which should go from `-u_max` to `+u_max`), the damping parameter $a$, and the number of points $n_\nu$. Create another function called `tau_scale()` that will calculate the optical depths scaled by the line profile and $\eta$, so that at the wings $\tau_0^l = \tau_c$ and at the line core, $\tau_{\nu 0}^l = \tau_c \eta$. It should take as arguments the continuum optical depth scale (a 1D array of depth points), the line profile (a 1D array of frequency points), and $\eta$, the ratio of line to continuum extinction. It should return a 2D array of $\tau_\nu^l$, where one dimension is depth and the second dimension is frequency.  
        
* *[12 points]* Write a non-LTE solver for the two-level case with complete redistribution, in a function called `solve_twolevel()`. As before, use ALI with the diagonal operator, in this case $\overline{\mathbf{\Lambda}}^*$. Make use of the `feautrier()` to obtain both $P_\mu\nu$ (and integrating over angle and frequency to obtain $\overline{J}$) and $L^*$, the diagonal of 
$\overline{\mathbf{\Lambda}}^*$. `solve_twolevel()` should take as arguments $B$ (1D array), $\varepsilon$ (1D array), $\varphi_\nu$ (1D array), and $\eta$ (scalar), and return S and $\overline{J}$. It should be used together with your previous functions `profile()` and `tau_scale()`.
    
* *[8 points]* Use `solve_twolevel()` for the FALC model atmosphere. Use the atmosphere's $\tau_{500}$ as $\tau_c$, and create a line profile with `u_max=5`, $a$=0.1, and 51 frequency points. The free parameters here will be $\varepsilon$ and $\eta$. Study the case where $\varepsilon$ is constant in the atmosphere. Adjust $\varepsilon$ and $\eta$ you obtain a line that thermalises in the photosphere ($\tau^c\approx 1$, line profile should be in absorption), and one that thermalises in the chromosphere ($\tau^c \approx 10^{-4}$, line profile should have emission in the core). Discuss. 
    
* *[6 points]* In real atmospheres, $\varepsilon$ varies with height. Assume $\eta=10^6$, compare a case with a constant $\varepsilon=10^{-3}$ with the case where $\varepsilon$ is given by `epsilon` below, which starts at 1 but then drops down to a minimum of $10^{-4}$. Why is $J$ at the surface higher in the constant $\varepsilon=10^{-3}$ , even though the second case has a lower value of $\varepsilon$ ($10^{-4}$) at the surface?

```python
tmp = numpy.linspace(-30, 10, len(falc['tau_500']))
epsilon = 1/(1 + numpy.exp(-(tmp) )) + 1e-4
```
</div>

***Hints:*** Do not use units with `feautrier`. You can obtain $B$ from FALC with units, but before passing any arrays to `feautrier` make sure they do not have units (e.g. take `.value` for $B$ and $\tau_{500}$).
    


In [4]:
# Your answers here